In [ ]:
# import pandas as pd

# logs_file_path = 'loginlogs.txt'
# logs_column_names = ['Date', 'Time', 'PC Name', 'User Name']
# df = pd.read_csv(logs_file_path, sep=' ; ', names=logs_column_names, engine='python')

# print(df.head())

In [ ]:
date_format_1 = '%d.%m.%Y'
date_format_2 = '%a %m/%d/%Y'
date_format_3 = '%d-%b-%Y'
date_format_4 = '%d-%b-%y'
date_format_5 = '%d/%m/%Y'
date_format_6 = '%a-%d-%b'

# Create a new column to store the original date values
df['Original_Date'] = df['Date']

# Attempt to parse the 'Date' column using each format
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', format=date_format_1).combine_first(
    pd.to_datetime(df['Date'], errors='coerce', format=date_format_2)).combine_first(
    pd.to_datetime(df['Date'], errors='coerce', format=date_format_3)).combine_first(
    pd.to_datetime(df['Date'], errors='coerce', format=date_format_4)).combine_first(
    pd.to_datetime(df['Date'], errors='coerce', format=date_format_5)).combine_first(
    pd.to_datetime(df['Date'], errors='coerce', format=date_format_6))

# Use boolean indexing to retrieve values from 'Original_Date' where 'Date' is 'NaT'
na_values = df.loc[df['Date'].isna(), 'Original_Date']

print(na_values)

In [121]:
import pandas as pd

# Step 1: Read both files into DataFrames
loginlogs_df = pd.read_csv("loginlogs.txt", sep=" ; ", header=None, names=["date", "time", "PC name", "user name"], engine='python')
school_pcs_df = pd.read_csv("School_PCs.txt", sep=" ; ", header=None, names=["Inventory number", "Asset name", "Asset type", "Location", "Responsible", "Position", "Year", "Comment"], engine='python')

# Step 2: Extract inventory numbers from School_PCs.txt
inventory_numbers = school_pcs_df["Inventory number"]

# Step 3: Filter loginlogs_df to include only rows with PC names containing inventory numbers
df = loginlogs_df[loginlogs_df["PC name"].str.contains('|'.join(map(str, inventory_numbers)), case=False)]

# Display the filtered DataFrame
print(df.head())

              date         time      PC name            user name
6   Fri 09/01/2023  10:31:20.62  011108499NU          abdul.wahab
10  Fri 09/01/2023  10:30:41.34  000553011NU     nazym.alipbayeva
12  Fri 09/01/2023  10:28:11.77  000580520NU  eldar.sharafutdinov
25  Fri 09/01/2023  10:20:22.48  000580519NU       aibek.shokayev
26  Fri 09/01/2023  10:19:15.87  NLT069054NU     kassym.talgatuly


In [122]:
# Create a new column to store the original date values
df.loc[:,'Original_Date'] = df['date'].copy()


date_format_1 = '%a %m/%d/%Y'
# date_format_2 = 

df.loc[:, 'date'] = pd.to_datetime(df['date'], errors='coerce', format=date_format_1)
df

C:\Users\user\AppData\Local\Temp\ipykernel_12048\1085964958.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'Original_Date'] = df['date'].copy()


,date,time,PC name,user name,Original_Date
6,2023-09-01 00:00:00,10:31:20.62,011108499NU,abdul.wahab,Fri 09/01/2023
10,2023-09-01 00:00:00,10:30:41.34,000553011NU,nazym.alipbayeva,Fri 09/01/2023
12,2023-09-01 00:00:00,10:28:11.77,000580520NU,eldar.sharafutdinov,Fri 09/01/2023
25,2023-09-01 00:00:00,10:20:22.48,000580519NU,aibek.shokayev,Fri 09/01/2023
26,2023-09-01 00:00:00,10:19:15.87,NLT069054NU,kassym.talgatuly,Fri 09/01/2023
...,...,...,...,...,...
352374,NaT,10:35:43.49,NLT066649NU,kanat.dukenbayev,16-Aug-21
352387,NaT,10:16:42.69,NLT063857NU,tomiris.madiyarova,16/08/2021
352393,2021-08-16 00:00:00,9:59:05.73,NLT042613,nursultan.jyeniskhan,Mon 08/16/2021
352394,2021-08-16 00:00:00,9:58:48.87,011107760NU,saule.mergenbayeva,Mon 08/16/2021


In [123]:
count = len(df['date'])
na_count = df['date'].isna().sum()
print(f'Percentage of NaT is: {round(na_count/count*100,2)}%')

Percentage of NaT is: 12.8%


In [113]:
df['date'].isna()

6         False
10        False
12        False
25        False
26        False
          ...  
352374     True
352387     True
352393    False
352394    False
352403    False
Name: date, Length: 75679, dtype: bool

In [124]:
date = pd.NaT
df.loc[df["date"] == date, "Original_Date"].iloc[0]

IndexError: single positional indexer is out-of-bounds

In [ ]:
inv_num = "NLT060945"
school_pcs_df.loc[school_pcs_df["Inventory number"] == inv_num, "Responsible"].iloc[0]

In [ ]:
filtered_loginlogs_df[filtered_loginlogs_df["user name"] == "abay.kasken"]

In [ ]:
inv_num = "NLT060945"
school_pcs_df[school_pcs_df["Inventory number"] == inv_num]

In [126]:
date = pd.Timestamp('2065-02-23 00:00:00')

df[df["date"] == date]

,date,time,PC name,user name,Original_Date
119752,2065-02-23 00:00:00,7:44:00.47,NLT042703NU,almas.turakov,Mon 02/23/2065


In [130]:
date_counts = df['date'].value_counts(dropna=False)
dict(date_counts)

{NaT: 9688,
 Timestamp('2023-08-29 00:00:00'): 418,
 Timestamp('2023-08-22 00:00:00'): 412,
 Timestamp('2023-08-28 00:00:00'): 408,
 Timestamp('2023-08-31 00:00:00'): 391,
 Timestamp('2023-04-13 00:00:00'): 389,
 Timestamp('2022-09-19 00:00:00'): 386,
 Timestamp('2023-08-21 00:00:00'): 371,
 Timestamp('2022-09-07 00:00:00'): 364,
 Timestamp('2022-09-06 00:00:00'): 363,
 Timestamp('2023-02-16 00:00:00'): 352,
 Timestamp('2023-02-02 00:00:00'): 351,
 Timestamp('2023-02-15 00:00:00'): 351,
 Timestamp('2022-09-22 00:00:00'): 345,
 Timestamp('2022-09-05 00:00:00'): 344,
 Timestamp('2022-10-10 00:00:00'): 340,
 Timestamp('2023-08-23 00:00:00'): 340,
 Timestamp('2022-09-09 00:00:00'): 338,
 Timestamp('2023-03-09 00:00:00'): 334,
 Timestamp('2023-01-31 00:00:00'): 334,
 Timestamp('2022-10-12 00:00:00'): 332,
 Timestamp('2022-09-15 00:00:00'): 328,
 Timestamp('2022-11-09 00:00:00'): 328,
 Timestamp('2022-08-25 00:00:00'): 326,
 Timestamp('2022-10-13 00:00:00'): 323,
 Timestamp('2023-02-06 00:00